# PETs/TETs – Hyperledger Aries – City (Holder) 🏙️

In [1]:
%%javascript
document.title ='🏙️ City Agent'

<IPython.core.display.Javascript object>

## PART 3: Connect with Manufacturers and Analyze Data

**What:** -

**Why:** -

**How:** <br>


**Accompanying Agents and Notebooks:**
* Manufacturer1 💼: `03_connect_with_cit.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
from aries_cloudcontroller import AriesAgentController
import libs.helpers as helpers
from libs.agent_connection_manager import RelyingParty
import os
import time
from termcolor import colored
from pprintpp import pprint

#### 0.2 – Variables

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate City Agent
#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://city-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [6]:
# The CredentialHolder registers relevant webhook servers and event listeners
city_agent = RelyingParty(agent_controller)

Successfully initiated AgentConnectionManager for a(n) RelyingParty ACA-PY agent


---

<a id=2></a>

### 2 – Establish a connection with the Authority agent
#### 2.1 Define VC Presentation Request Object

The below cell defines a generic presentation request object, that can be sent across specific connections requesting that they produce a presentation containing the identified attributes and meeting the restrictions.

It is often useful to define your request objects first, then reuse these objects across many connections you wish to request a proof from. 

Duplicate and customise the below cell as many times as you need. It may be useful to save these request objects either to the jupyter store using %store or through

TODO: Detail the full set of restrictions available to a verifier.

In [7]:
# List of required attribuets and restrictions
identifiers = helpers.get_identifiers()
schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
req_attrs = [
    {"name": "isManufacturer", "restrictions": [{"schema_id": schema_manufacturer_id}]},
]

# Define proof_requeset
manufacturer_proof_request = {
    "name": "isManufacturer Proof Request",
    "version": "1.0",
    "requested_attributes": {f"0_{req_attr['name']}_uuid": req_attr for req_attr in req_attrs },
    # Ignore predicates (e.g., range proofs)
    "requested_predicates": {}, #{f"0_{req_pred['name']}_GE_uuid": req_pred for req_pred in req_preds },
    # You can also request the entire proof request be non-revoked
    "non_revoked":  {"to": int(time.time())}
}

print(colored("Manufacturer Proof Request:", attrs=["bold"]))
pprint(manufacturer_proof_request)

Manufacturer Proof Request:
{
    'name': 'isManufacturer Proof Request',
    'non_revoked': {'to': 1629291522},
    'requested_attributes': {
        '0_isManufacturer_uuid': {
            'name': 'isManufacturer',
            'restrictions': [
                {
                    'schema_id': 'au8Y1rjsZyz9jbpr3Lk54:2:certify-manufacturer:0.0.1',
                },
            ],
        },
    },
    'requested_predicates': {},
    'version': '1.0',
}


#### 2.2 Create multi-use invitation
Send out a multi-use invitation to all manufacturer agents (i.e., copy & paste the same invitation to manufacturer1, manufacturer2, manufacturer3). This represents a scenario where the City agent invites any agent to connect with them, and authenticate as manufacturers. The advantage of a multi-use invitation is, that the City is unaware about who accessed the invitation link and is trying to get in contact with the City.

**Note:** Please establish a connection with all three manufacturer agents.

In [15]:
# Setup for connection with Authority agent
alias = None
auto_accept = True # Accept response of Authority agent right away
auto_ping = True
public = False # Do not use public DID
multi_use = True # Invitation is only for one invitee

invitation = city_agent.create_connection_invitation(alias=alias, auto_accept=auto_accept, public=public, multi_use=multi_use, auto_ping=auto_ping)


---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  80fd6fa7-aa9f-41cb-a838-25a1546903d0
State :  invitation (invitation-sent)
Routing State : none
Their Role :  invitee
---------------------------------------------------------------------

Copy & paste invitation and share with external agent:
{
    '@id': '71dff343-d2ca-46e4-82bf-c04774941664',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['CBUT7cYooo3F39f7P3LgdqZABppWcjHKCdP25z5wJNmN'],
    'serviceEndpoint': 'https://898696e3e9e9.ngrok.io',
}

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  55a2b2b3-0259-442b-8b62-1c54dfa77212
State :  invitation (invitation-sent)
Routing State : none
Their Role :  invitee
---------------------------------------------------------------------

-----------------------------------------

<div style="font-size: 25px"><center><b>Break Point 1</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 💼💼💼</center></div><br>
<center><b>Please open all manufacturer agents 💼💼💼. <br> For each of the manufacturer agents, open the 03_connect_with_city.ipynb notebook and execute all steps until Break Point 2/3/4. <br> Use the same invitation from the City Agent 🏙️ in Step 2.1 for all Manufacturers.</b></center>


#### 2.3 Display all active connections

In [12]:
# Display all active connections
for conn in city_agent.get_active_connections():
    conn.display()


---------------------------------------------------------------------
Connection with Manufacturer1
Connection ID :  3410b239-a3ad-4e06-b7bd-3c18b24154d4
Connection with :  Manufacturer1
Is Active :  True
Auto Ping :  False
Auto Accept :  True
---------------------------------------------------------------------


#### 2.4 Fetch all connection_ids of the active connections with the respective manufacturer agents

In [14]:
# Get conneciton_id with Manufacturer1
connection_id_m1 = city_agent.get_connection_id("Manufacturer1")[0] # We assume that there is only one connection with Manufacturer1

# Get conneciton_id with Manufacturer2
#connection_id_m1 = city_agent.get_connection_id("Manufacturer2")[0] # We assume that there is only one connection with Manufacturer2

# Get conneciton_id with Manufacturer3
#connection_id_m1 = city_agent.get_connection_id("Manufacturer3")[0] # We assume that there is only one connection with Manufacturer3

'3410b239-a3ad-4e06-b7bd-3c18b24154d4'

### 3 – Send Proof Request

This identifies a connection and a proof request object (you will need to update the variable name enclosed with <>) 

In [ ]:
# Step 1

proof_request = {
    "comment": "Please prove that you are an agent who is a certified manufacturer",
    "connection_id": connection_id_m1,
    "proof_request": manufacturer_proof_request,
    "trace": False
}

proof_request_response = await agent_controller.proofs.send_request(proof_request)

*I THINK* **Break point**: see step 2

---

## Get Presentation Exchange Record

This record keeps track of the current state of the presentation protocol, which must be in the `presentation_received` state before the presentation can be verified.

Note: This could also happen in the webhook logic.

In [ ]:
presentation_exchange_id = proof_request_response["presentation_exchange_id"]
pres_record = await agent_controller.proofs.get_record_by_id(presentation_exchange_id)
print(pres_record)

## Verify Presentation

Only if it is in the right state. 

Note: Verifying a presentation moves the state to `verified` regardless of whether the presentation request has been satisfied. To check this you must refer to the `verified` property on the response.

In [ ]:
verified_response = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
verified = verified_response["verified"]

## Parsing Disclosed Attribute Values from Presentation

A presentation object contains three classes of attributes. 
* Revealed Attributes: Attributes that were signed by an issuer and have been revealed in the presentation process
* Self Attested Attributes: Attributes that the prover has self attested to in the presentation object.
* Predicate proofs: Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

### Parse Revealed Attributes

In [ ]:
for (name, val) in verified_response['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print("\nAttribute : ", val)
    
    attr_name = verified_response["presentation_request"]["requested_attributes"][name]["name"]
    print("Attribute Name :  Raw Value")
    print(f"{attr_name}   :  {val['raw']}")

### Parse Self-Attested Attributes

In [ ]:
for (name, val) in verified_response['presentation']['requested_proof']['self_attested_attrs'].items():
    print(name)
    ## Slightly different for self attested attrs
    print(val)

## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [6]:
# Imports
import torch
import syft as sy

from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

__init___: register primitive generator
__init___: register generator
__init___: register primitive store add
__init___: register add
__init___: register primitive store get
__init___: register get
__init___: register primitive generator
__init___: register generator
__init___: register primitive store add
__init___: register add
__init___: register primitive store get
__init___: register get
__init___: register primitive generator
__init___: register generator
__init___: register primitive store add
__init___: register add
__init___: register primitive store get
__init___: register get
__init___: register primitive generator
__init___: register generator
__init___: register primitive store add
__init___: register add
__init___: register primitive store get
__init___: register get
__init___: register primitive generator
__init___: register generator
__init___: register primitive generator
__init___: register generator
__init___: register primitive generator
__init___: register generato

In [9]:
# Step 4: Connect to duet of M1
duet_m1 = sy.duet("86a6992e42ce977bbe6e9bf698d25a72")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 4a55d85bf23402d200453e73320cf022

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [ ]:
print(f"duet id: {duet_m1.id}, duet target it: {duet_m1.target_id}, duet note: {duet_m1.node}")

In [10]:
duet_m1.store.pandas

""


In [ ]:
from IPython.core.debugger import set_trace

duet_m1.store.pandas

## 2 - Secure Multi-Party Computation

In [11]:
# Step 5: Setup session with all duets
session = Session(parties=[duet_m1])
print(session)

In [12]:
# Step 6: Setup MPC session
SessionManager.setup_mpc(session)

In [14]:
#duet_ds.store.pandas

In [16]:
#from IPython.core.debugger import set_trace

# Step 7: NOW we're ready to set up private operaitions
#%pdb
duet_m1.store.pandas

,ID,Tags,Description,object_type
0,<UID: df707aa82e434c1b8a3b6b203f0afa8c>,[],,<class 'sympc.session.session.Session'>
1,<UID: 850a970ea3cd408196d64225862d6103>,[test_data_age],description for age_data,<class 'torch.Tensor'>


In [17]:
# Step 12: basic operations
x_secret = duet_m1.store["test_data_age"] # describe local data to test sum, substract, and multiply

In [18]:
y = torch.Tensor([[-5,8,1,7,6,100]]) # Local data

In [19]:
x = MPCTensor(secret=x_secret, shape=(1,), session=session)  # MPC Tensor from x_secret

The following lines of codes always receive an error message.
[See Github issue](https://github.com/OpenMined/SyMPC/issues/282).

In [ ]:
print("X + Y = ",(x + y).reconstruct())

In [ ]:
print("X - Y = ", (x - y).reconstruct())

In [ ]:
print("X * Y = ", (x * y).reconstruct())

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()